# Get candidate triplets

Retrieve and display the triplets (new, ref, diff) for a candidate by name.

This requires access to the private databases at NERSC.

## Set Up

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
from IPython.display import display
import ipywidgets

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

from copy import deepcopy
import time

In [ ]:
font = {'size' : 15}
mpl.rc('font', **font)

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
db.autocommit = True # Makes the computer nicer about SQL syntax errors

In [ ]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

## Choose a candidate and view all triplets

In [ ]:
# use_candid = 'DC21ksmv'
# use_candid = 'DC21bgiwr'
# use_candid = 'DC21qcpw'

### exploring ELAIS candidates with correlated brightness and R/B score
# use_candid = 'DC21kkqi'
# use_candid = 'DC21knug'
use_candid = 'DC21klix'

In [ ]:
query = ("SELECT o.id, o.ra, o.dec, o.candidate_id, o.subtraction_id, rbs.rb, o.mag, o.magerr, o.ignore FROM objects o "
         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
         "WHERE o.candidate_id = %s")

cursor.execute( query, (use_candid, ) )

results = np.array( cursor.fetchall() ).transpose()

In [ ]:
all_obj_ids = results[0]

In [ ]:
print(all_obj_ids)

In [ ]:
for objid in all_obj_ids:
    
    query = ( 'SELECT object_id, sci_jpeg, ref_jpeg, diff_jpeg FROM cutouts '
              'WHERE object_id = '+objid)
    cursor.execute( query )

    for row in cursor:
        string = '<b>Candidate: </b>'+use_candid+'<br>'
        string += '<b>Object: </b>'+str(row["object_id"])+'<br>'
        label = ipywidgets.HTML( string )
        newimg = ipywidgets.Image( value=bytes(row["sci_jpeg"]), width=102, height=102, format='jpeg' )
        refimg = ipywidgets.Image( value=bytes(row["ref_jpeg"]), width=102, height=102, format='jpeg' )
        subimg = ipywidgets.Image( value=bytes(row["diff_jpeg"]), width=102, height=102, format='jpeg' )
        box = ipywidgets.HBox( [ newimg, refimg, subimg, label ] )
        display( box )

#### this version is not working for some reason, tbd

In [ ]:
# for use_objid in all_obj_ids:

#     query = ( 'SELECT e.mjd, e.filename, e.filter, c.object_id, c.sci_jpeg, c.ref_jpeg, c.diff_jpeg FROM cutouts c '
#               'INNER JOIN objects o ON c.object_id = o.id '
#               'INNER JOIN subtractions s ON o.subtraction_id = s.id '
#               'INNER JOIN exposures e ON s.exposure_id = e.id '
#               'WHERE c.object_id = %s AND e.mjd > 59725 AND e.filter = %s')

#     cursor.execute( query, (use_objid, 'g' ) )
    
#     for row in cursor:
#         label = ipywidgets.HTML( f'<b>Candidate:</b> {use_candid}<br><b>MJD:</b> {row["mjd"]}<br><b>Filter:</b> {row["filter"]}' )
#         newimg = ipywidgets.Image( value=bytes(row["sci_jpeg"]), width=102, height=102, format='jpeg' )
#         refimg = ipywidgets.Image( value=bytes(row["ref_jpeg"]), width=102, height=102, format='jpeg' )
#         subimg = ipywidgets.Image( value=bytes(row["diff_jpeg"]), width=102, height=102, format='jpeg' )
#         box = ipywidgets.HBox( [ newimg, refimg, subimg, label ] )
#         display( box )